# Notebook for F1.5 Calculator Testing


## Library Importing

In [1]:
import pandas as pd

## setup data

In [2]:
race_data_address = "Formula1_2023season_raceResults.csv"
sprint_data_address = ""


In [3]:
race_data = pd.read_csv(race_data_address)
race_data.head()

,Track,Position,No,Driver,Team,Starting Grid,Laps,Time/Retired,Points,Set Fastest Lap,Fastest Lap Time
0,Bahrain,1,1,Max Verstappen,Red Bull Racing Honda RBPT,1,57,1:33:56.736,25,No,1:36.236
1,Bahrain,2,11,Sergio Perez,Red Bull Racing Honda RBPT,2,57,+11.987,18,No,1:36.344
2,Bahrain,3,14,Fernando Alonso,Aston Martin Aramco Mercedes,5,57,+38.637,15,No,1:36.156
3,Bahrain,4,55,Carlos Sainz,Ferrari,4,57,+48.052,12,No,1:37.130
4,Bahrain,5,44,Lewis Hamilton,Mercedes,7,57,+50.977,10,No,1:36.546


## Getting All tracks, drivers and teams in the dataset

In [4]:
banned_teams =['Red Bull Racing Honda RBPT']

all_races = list( dict.fromkeys(race_data['Track']) )
all_teams = list( dict.fromkeys(race_data[(~race_data['Team'].isin(banned_teams))]['Team']) )
all_drivers = list( dict.fromkeys(race_data[(~race_data['Team'].isin(banned_teams))]['Driver']) )
all_teams 

['Aston Martin Aramco Mercedes',
 'Ferrari',
 'Mercedes',
 'Alfa Romeo Ferrari',
 'Alpine Renault',
 'Williams Mercedes',
 'AlphaTauri Honda RBPT',
 'Haas Ferrari',
 'McLaren Mercedes',
 'AlphaTauri Honda RPBT']

## Create Driver Scoring Table

In [5]:
class standings:
    def __init__(self):
        temp_data= {'Driver': all_drivers,
                    'Points':[0]*len(all_drivers)}
        self.driverTable= pd.DataFrame(temp_data).set_index('Driver')
        
        temp_data= {'Team': all_teams,
                    'Points':[0]*len(all_teams)}
        self.constructorTable= pd.DataFrame(temp_data).set_index('Team')
        
        
    def add_to_standings(self,data):
        for i in data.index:
#             print(i)
#             print(data['Points'][i])
            self.driverTable['Points'][i]+=data['Points'][i]
            self.constructorTable['Points'][data['Team'][i]]+=data['Points'][i]
    
        self.driverTable=self.driverTable.sort_values('Points',ascending=False)
        self.constructorTable=self.constructorTable.sort_values('Points',ascending=False)

                             
            

## Getting Race data for a specific track and excluding banned teams

In [6]:


interesting_columns = ['Track','Position','Driver','Team','Points','Fastest Lap Time']

def get_race_data(race):
    _specific_race_data =race_data[(race_data['Track']== race) &(~race_data['Team'].isin(banned_teams))].filter(interesting_columns).copy()
    
    return _specific_race_data

#specific_race_data = get_race_data(all_races[1])
#specific_race_data

## Redo positions

In [7]:
def redo_positions(_specific_race_data):
    
    not_classified_count = _specific_race_data['Position'].tolist().count('NC')
    drivers_in_race = len(_specific_race_data['Position']) - not_classified_count
    new_positions = list(range(1,drivers_in_race+1)) + (['NC'] * not_classified_count)
    _specific_race_data['Position']=new_positions
    
    return  _specific_race_data.copy()

#specific_race_data = redo_positions(specific_race_data)


## Get Fastest Lap Driver

In [8]:
def get_fastest_driver(_specific_race_data):
    fastest_lap = _specific_race_data['Fastest Lap Time'].sort_values().iloc[0]
    _fastest_driver = _specific_race_data[(_specific_race_data['Fastest Lap Time'] == fastest_lap)]
    return _fastest_driver
#fastest_driver = get_fastest_driver(specific_race_data)

## Get Points per driver

In [9]:
# This should be adjusted for sprints where points are different
# The 100 at the end creates 100 0 point positions meaning this in theory supports 110 drivers before list size error

points_race = [25,18,15,12,10,8,6,4,2,1]+([0]*100)


def make_scoreboard(_specific_race_data, _fastest_driver):
    not_classified_count = _specific_race_data['Position'].tolist().count('NC')
    drivers_in_race = len(_specific_race_data['Position']) - not_classified_count
    new_points = [0] * (not_classified_count + drivers_in_race)


    # add points for all classified drivers
    for i in range(drivers_in_race):
        position = _specific_race_data.iloc[i]['Position']
        new_points[i] = points_race[position-1]

    


    if(isinstance(_fastest_driver['Position'].iloc[0],int)   and _fastest_driver['Position'].iloc[0]<=10 ):
        new_points[_fastest_driver['Position'].iloc[0]-1] +=1


    _specific_race_data['Points'] = new_points
    _specific_race_data = _specific_race_data.copy()
    return _specific_race_data
#specific_race_data = make_scoreboard(specific_race_data)

## Function for getting scoreboard per race

In [10]:
def scoreboard(race):
    _race_data= get_race_data(race)
    _race_data = redo_positions(_race_data)
    __fastest_driver = get_fastest_driver(_race_data)
    return make_scoreboard(_race_data,__fastest_driver).filter(['Driver','Team','Points']).set_index('Driver')




In [12]:
DriverTable = standings()

for i in all_races:
    DriverTable.add_to_standings(scoreboard(i))


DriverTable.constructorTable

,Points
Team,
Mercedes,409
Ferrari,377
Aston Martin Aramco Mercedes,331
McLaren Mercedes,241
Alpine Renault,133
Williams Mercedes,50
Alfa Romeo Ferrari,36
AlphaTauri Honda RBPT,28
Haas Ferrari,24
